## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,8/6/20,Down ES Up WS to GWB,28.47,3.0,10.0,3.17,8.99,Specialized
1,9/14/16,Citibike to Work,1.09,NaN,11.0,0.18,5.95,Citibike
2,8/26/19,Down ES Up UWS,21.90,2.0,55.0,2.92,7.51,Specialized
3,9/8/19,Wards Island,11.43,1.0,32.0,1.53,7.45,Specialized
4,8/27/17,CP BL,6.75,NaN,43.0,0.72,9.42,Specialized
...,...,...,...,...,...,...,...,...
350,3/9/21,Dn ES Up LWS,15.90,2.0,20.0,2.33,6.81,Specialized
351,7/24/19,Down ES Up WS,16.29,2.0,36.0,2.60,6.27,Specialized
352,11/28/19,Down ES up ES,14.54,1.0,48.0,1.80,8.08,Specialized
353,9/4/20,To Astor Wines,2.92,NaN,25.0,0.42,7.01,Citibike


In [5]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,8/6/20,Down ES Up WS to GWB,28.47,3.0,10.0,3.17,8.99,Specialized
1,9/14/16,Citibike to Work,1.09,0.0,11.0,0.18,5.95,Citibike
2,8/26/19,Down ES Up UWS,21.90,2.0,55.0,2.92,7.51,Specialized
3,9/8/19,Wards Island,11.43,1.0,32.0,1.53,7.45,Specialized
4,8/27/17,CP BL,6.75,0.0,43.0,0.72,9.42,Specialized
...,...,...,...,...,...,...,...,...
350,3/9/21,Dn ES Up LWS,15.90,2.0,20.0,2.33,6.81,Specialized
351,7/24/19,Down ES Up WS,16.29,2.0,36.0,2.60,6.27,Specialized
352,11/28/19,Down ES up ES,14.54,1.0,48.0,1.80,8.08,Specialized
353,9/4/20,To Astor Wines,2.92,0.0,25.0,0.42,7.01,Citibike


In [6]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,8/6/20,Down ES Up WS to GWB,28.47,3.0,10.0,3.17,8.99,Specialized
1,1,9/14/16,Citibike to Work,1.09,0.0,11.0,0.18,5.95,Citibike
2,2,8/26/19,Down ES Up UWS,21.90,2.0,55.0,2.92,7.51,Specialized
3,3,9/8/19,Wards Island,11.43,1.0,32.0,1.53,7.45,Specialized
4,4,8/27/17,CP BL,6.75,0.0,43.0,0.72,9.42,Specialized
...,...,...,...,...,...,...,...,...,...
350,350,3/9/21,Dn ES Up LWS,15.90,2.0,20.0,2.33,6.81,Specialized
351,351,7/24/19,Down ES Up WS,16.29,2.0,36.0,2.60,6.27,Specialized
352,352,11/28/19,Down ES up ES,14.54,1.0,48.0,1.80,8.08,Specialized
353,353,9/4/20,To Astor Wines,2.92,0.0,25.0,0.42,7.01,Citibike


In [7]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [8]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,28.47,3.17,8.99,Specialized
1,1.09,0.18,5.95,Citibike
2,21.90,2.92,7.51,Specialized
3,11.43,1.53,7.45,Specialized
4,6.75,0.72,9.42,Specialized
...,...,...,...,...
350,15.90,2.33,6.81,Specialized
351,16.29,2.60,6.27,Specialized
352,14.54,1.80,8.08,Specialized
353,2.92,0.42,7.01,Citibike


In [9]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [10]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [11]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [12]:
# Checking training data shape
X_train.shape

(266, 3)

In [13]:
# Checking testing data shape
X_test.shape

(89, 3)

In [14]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [15]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [16]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [17]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9887640449438202


In [18]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Specialized
1,Citibike
2,Specialized
3,Specialized
4,Specialized
...,...
84,Citibike
85,Citibike
86,Specialized
87,Citibike


In [19]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
107,Specialized
302,Citibike
303,Specialized
58,Specialized
147,Specialized
...,...
323,Citibike
221,Citibike
131,Specialized
167,Citibike


In [20]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,107,Specialized
1,302,Citibike
2,303,Specialized
3,58,Specialized
4,147,Specialized
...,...,...
84,323,Citibike
85,221,Citibike
86,131,Specialized
87,167,Citibike


In [21]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,107,Specialized,Specialized
1,302,Citibike,Citibike
2,303,Specialized,Specialized
3,58,Specialized,Specialized
4,147,Specialized,Specialized
...,...,...,...
84,323,Citibike,Citibike
85,221,Citibike,Citibike
86,131,Specialized,Specialized
87,167,Citibike,Citibike


In [22]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
72,94,Specialized,Citibike


In [23]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,94,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.8,Specialized


In [24]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,94,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.8
